In [479]:
from bs4 import BeautifulSoup
import math
import lxml
import os
import json
import re
import snappy
from PorterStemmer import PorterStemmer
import numpy as np
import pandas as pd
import nltk
import random
import sys

In [480]:
queries={}
dict_to_vec={}
vec_to_dict={}
dictionary={}
doc_scores={}
dict_to_vec={}
vec_to_dict={}
smoothing_vector={}
mu=1000
relevance_model='rm1'


In [481]:
directory= r"C:\Users\HP\Desktop\Study\COL764\Project\AILA_2019_dataset\Object_statutes"

In [482]:
def stemmer(tokenizedText):
    ps=PorterStemmer()
    stemmedText=[]
    for word in tokenizedText:
        stemmedText.append(ps.stem(word.lower(),0,len(word)-1))
    return stemmedText
def tokenizer(docText):
    tokenizedText=re.split(' |;|,|\*|\n|:|\(|\)|`|\.|\{|\}|\'|\"|\[|\]|\_|>|^|<|\t|=|#',docText)
    tokenizedText=set(tokenizedText)
    if '' in tokenizedText:
        tokenizedText.remove('')
    return tokenizedText
def tokenizerForDocs(docText):
    tokenizedText=re.split(' |;|,|\*|\n|:|\(|\)|`|\.|\{|\}|\'|\"|\[|\]|\_|>|^|<|\t|=|#',docText)
    return tokenizedText

In [483]:
def createQueryDictionary(queryWords):
    dictionary={}
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            with open(subdir+'/'+file,mode="r", encoding = "utf8", errors="ignore") as f:
                doc_curr=f.read()
                tokenizedText=tokenizer(doc_curr)
                stemmedText=stemmer(tokenizedText)
                stemmedText=set(stemmedText)
                for word in stemmedText:
                    if not word in dictionary:
                        dictionary[word]=1
    for word in queryWords:
        if not word in dictionary:
                dictionary[word]=1
    return dictionary

In [484]:
def generateProbabilities(queryWords):
    collection_vec=np.ones(len(dictionary))
    final_vec=np.zeros(len(dictionary))
    num_words_doc=0
    num_words_collection=0
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            with open(subdir+'/'+file,mode="r", encoding = "utf8", errors="ignore") as f:
                doc_curr=f.read()
                tokenizedText=tokenizerForDocs(doc_curr)
                stemmedText=stemmer(tokenizedText)
                curr_vec=np.zeros(len(dictionary))
                for word in stemmedText:
                    if word in dictionary:
                        curr_vec[dict_to_vec[word]]+=1
                        num_words_doc+=1
                doc_scores[str(file[:-4])]=curr_vec
                doc_num_words[str(file[:-4])]=num_words_doc
                num_words_collection+=num_words_doc
                num_words_doc=0
                collection_vec+=curr_vec
    collection_vec=collection_vec/num_words_collection
    for cord_id in doc_scores.keys():
        doc_scores[cord_id]=(doc_scores[cord_id]+(mu*collection_vec))/(doc_num_words[cord_id]+mu)
    if relevance_model=='rm1':
        for cord_id in doc_scores.keys():
            multiplier=1e200
            for word in queryWords :
                if word in dictionary:
                    multiplier*=doc_scores[cord_id][dict_to_vec[word]]*1e3
            doc_scores[cord_id]=doc_scores[cord_id]*multiplier
            final_vec+=doc_scores[cord_id]
    if relevance_model=='rm2':
        pw_vector=np.zeros(len(dictionary))
        query_vectors={}
        for cord_id in doc_scores.keys():
            pw_vector+=doc_scores[cord_id]
        final_vec=np.ones(len(dictionary))
        final_vec*=pw_vector
        for word in queryWords:
            word_vec=np.zeros(len(dictionary))
            for cord_id in doc_scores.keys():
                word_vec+=(pw_vector*doc_scores[cord_id]*doc_scores[cord_id][dict_to_vec[word]])
            final_vec*=word_vec
    print(final_vec.sum())      
    return final_vec

In [485]:
def wordRanker(final_vec):
    reRanked=pd.DataFrame({'word':[],'probability':[]})
    for i in range(len(final_vec)):
        if not vec_to_dict[i] in nltk_stopwords:
            reRanked=reRanked.append({'word':vec_to_dict[i] ,'probability':final_vec[i]},ignore_index=True)
    return reRanked.sort_values(by=['probability'], ascending=False)

In [486]:
nltk_stopwords=["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would", "able", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "afterwards", "ah", "almost", "alone", "along", "already", "also", "although", "always", "among", "amongst", "announce", "another", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "arent", "arise", "around", "aside", "ask", "asking", "auth", "available", "away", "awfully", "b", "back", "became", "become", "becomes", "becoming", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "believe", "beside", "besides", "beyond", "biol", "brief", "briefly", "c", "ca", "came", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "couldnt", "date", "different", "done", "downwards", "due", "e", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "former", "formerly", "forth", "found", "four", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "happens", "hardly", "hed", "hence", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hi", "hid", "hither", "home", "howbeit", "however", "hundred", "id", "ie", "im", "immediate", "immediately", "importance", "important", "inc", "indeed", "index", "information", "instead", "invention", "inward", "itd", "it'll", "j", "k", "keep", "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "made", "mainly", "make", "makes", "many", "may", "maybe", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "moreover", "mostly", "mr", "mrs", "much", "mug", "must", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "nobody", "non", "none", "nonetheless", "noone", "normally", "nos", "noted", "nothing", "nowhere", "obtain", "obtained", "obviously", "often", "oh", "ok", "okay", "old", "omitted", "one", "ones", "onto", "ord", "others", "otherwise", "outside", "overall", "owing", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "said", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "shed", "shes", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "sufficiently", "suggest", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th", "thank", "thanks", "thanx", "thats", "that've", "thence", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "theyd", "theyre", "think", "thou", "though", "thoughh", "thousand", "throug", "throughout", "thru", "thus", "til", "tip", "together", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "unfortunately", "unless", "unlike", "unlikely", "unto", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "wasnt", "way", "wed", "welcome", "went", "werent", "whatever", "what'll", "whats", "whence", "whenever", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "whim", "whither", "whod", "whoever", "whole", "who'll", "whomever", "whos", "whose", "widely", "willing", "wish", "within", "without", "wont", "words", "world", "wouldnt", "www", "x", "yes", "yet", "youd", "youre", "z", "zero", "a's", "ain't", "allow", "allows", "apart", "appear", "appreciate", "appropriate", "associated", "best", "better", "c'mon", "c's", "cant", "changes", "clearly", "concerning", "consequently", "consider", "considering", "corresponding", "course", "currently", "definitely", "described", "despite", "entirely", "exactly", "example", "going", "greetings", "hello", "help", "hopefully", "ignored", "inasmuch", "indicate", "indicated", "indicates", "inner", "insofar", "it'd", "keep", "keeps", "novel", "presumably", "reasonably", "second", "secondly", "sensible", "serious", "seriously", "sure", "t's", "third", "thorough", "thoroughly", "three", "well", "wonder", "a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "co", "op", "research-articl", "pagecount", "cit", "ibid", "les", "le", "au", "que", "est", "pas", "vol", "el", "los", "pp", "u201d", "well-b", "http", "volumtype", "par", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a1", "a2", "a3", "a4", "ab", "ac", "ad", "ae", "af", "ag", "aj", "al", "an", "ao", "ap", "ar", "av", "aw", "ax", "ay", "az", "b1", "b2", "b3", "ba", "bc", "bd", "be", "bi", "bj", "bk", "bl", "bn", "bp", "br", "bs", "bt", "bu", "bx", "c1", "c2", "c3", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "cl", "cm", "cn", "cp", "cq", "cr", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d2", "da", "dc", "dd", "de", "df", "di", "dj", "dk", "dl", "do", "dp", "dr", "ds", "dt", "du", "dx", "dy", "e2", "e3", "ea", "ec", "ed", "ee", "ef", "ei", "ej", "el", "em", "en", "eo", "ep", "eq", "er", "es", "et", "eu", "ev", "ex", "ey", "f2", "fa", "fc", "ff", "fi", "fj", "fl", "fn", "fo", "fr", "fs", "ft", "fu", "fy", "ga", "ge", "gi", "gj", "gl", "go", "gr", "gs", "gy", "h2", "h3", "hh", "hi", "hj", "ho", "hr", "hs", "hu", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ic", "ie", "ig", "ih", "ii", "ij", "il", "in", "io", "ip", "iq", "ir", "iv", "ix", "iy", "iz", "jj", "jr", "js", "jt", "ju", "ke", "kg", "kj", "km", "ko", "l2", "la", "lb", "lc", "lf", "lj", "ln", "lo", "lr", "ls", "lt", "m2", "ml", "mn", "mo", "ms", "mt", "mu", "n2", "nc", "nd", "ne", "ng", "ni", "nj", "nl", "nn", "nr", "ns", "nt", "ny", "oa", "ob", "oc", "od", "of", "og", "oi", "oj", "ol", "om", "on", "oo", "oq", "or", "os", "ot", "ou", "ow", "ox", "oz", "p1", "p2", "p3", "pc", "pd", "pe", "pf", "ph", "pi", "pj", "pk", "pl", "pm", "pn", "po", "pq", "pr", "ps", "pt", "pu", "py", "qj", "qu", "r2", "ra", "rc", "rd", "rf", "rh", "ri", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "rv", "ry", "s2", "sa", "sc", "sd", "se", "sf", "si", "sj", "sl", "sm", "sn", "sp", "sq", "sr", "ss", "st", "sy", "sz", "t1", "t2", "t3", "tb", "tc", "td", "te", "tf", "th", "ti", "tj", "tl", "tm", "tn", "tp", "tq", "tr", "ts", "tt", "tv", "tx", "ue", "ui", "uj", "uk", "um", "un", "uo", "ur", "ut", "va", "wa", "vd", "wi", "vj", "vo", "wo", "vq", "vt", "vu", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y2", "yj", "yl", "yr", "ys", "yt", "zi", "zz"]


In [487]:

f=open('AILA_2019_dataset/Query_doc.txt','r')

query_file = f.read()

temp_queries = query_file.split("\n")

for query in temp_queries:
    queries[query.split("||")[0]]= query.split("||")[1]


In [488]:
out_dict = {}
for query in queries.keys():
    dict_to_vec={}
    vec_to_dict={}
    dictionary={}
    doc_scores={}
    doc_num_words={}
    dict_to_vec={}
    vec_to_dict={}
    tokenizedQuery=tokenizer(queries[query])
    queryWords=set(stemmer(tokenizedQuery))
    dictionary=createQueryDictionary(queryWords)
    temp2=0
    for key in dictionary.keys():
        dict_to_vec[key]=temp2
        vec_to_dict[temp2]=key
        temp2+=1
    final_vec=generateProbabilities(queryWords)
    top_20=wordRanker(final_vec)[:40]
    print(top_20)
    reRanked=pd.DataFrame({'cord_id':[],'kl_divergence_score':[]})
    for cord_id in doc_scores:
        kl_divergence_score=0
        for idx, row in top_20.iterrows():
            if not doc_scores[cord_id][dict_to_vec[row['word']]]==0:
                kl_divergence_score+=row['probability']*(math.log(doc_scores[cord_id][dict_to_vec[row['word']]]))
        reRanked=reRanked.append({'cord_id':cord_id ,'kl_divergence_score':kl_divergence_score},ignore_index=True)
    reRanked=reRanked.sort_values(by=['kl_divergence_score'], ascending=False)   

    out_dict[query]=[]
    for idx, row in reRanked.iterrows():
        out_dict[query].append((row['cord_id'],row['kl_divergence_score']))
#         result.write(query+' '+'Q0'+' '+row['cord_id']+' '+str(count)+' '+str(row['kl_divergence_score'])+' '+'abc'+'\n')
#         count+=1
#     for idx,row in top_20.iterrows():
#         top_words.write(query+' '+row['word']+'\n')
        

72935.38012859698
          word  probability
10         ani  1317.151154
72       court   560.182613
42         mai   559.676496
8        order   428.140048
41           2   401.458061
51      person   374.258924
46           1   360.971910
37         thi   344.639411
267   sub-sect   342.875597
174      state   320.802176
13      govern   304.970739
80       claus   278.263799
128  substitut   271.375223
33           3   255.864459
18        titl   246.560291
39        desc   237.945327
88         law   234.336591
69          ha   224.850963
63        case   218.890454
74       power   206.991939
122      amend   205.039403
105     insert   185.264198
127     provid   178.341221
2       applic   176.156023
156  constitut   170.579987
798      india   162.067269
240      refer   157.653879
22      articl   151.623451
85        year   149.889907
116     public   149.041019
424     punish   148.439410
56           4   146.454461
84     proceed   143.008673
140       word   141.285422
76

4.4967979315836746e+29
          word   probability
10         ani  7.856579e+27
42         mai  3.347034e+27
72       court  2.781309e+27
51      person  2.637638e+27
41           2  2.497540e+27
46           1  2.247734e+27
37         thi  2.148643e+27
267   sub-sect  2.132342e+27
174      state  2.019896e+27
8        order  1.930944e+27
13      govern  1.896568e+27
80       claus  1.730860e+27
128  substitut  1.692631e+27
88         law  1.669330e+27
33           3  1.591294e+27
18        titl  1.533237e+27
39        desc  1.479665e+27
69          ha  1.399820e+27
63        case  1.365180e+27
424     punish  1.322429e+27
122      amend  1.279112e+27
105     insert  1.154154e+27
127     provid  1.114653e+27
85        year  1.085855e+27
156  constitut  1.063454e+27
798      india  1.025577e+27
240      refer  9.803760e+26
421   imprison  9.595599e+26
22      articl  9.428754e+26
116     public  9.270911e+26
56           4  9.141488e+26
84     proceed  8.893180e+26
140       word  8.78

2.7429291983158814e+29
          word   probability
10         ani  4.771382e+27
42         mai  2.062580e+27
72       court  1.659563e+27
41           2  1.514788e+27
51      person  1.496250e+27
174      state  1.403672e+27
46           1  1.380179e+27
37         thi  1.297709e+27
267   sub-sect  1.290628e+27
8        order  1.157873e+27
13      govern  1.148122e+27
80       claus  1.047424e+27
128  substitut  1.039275e+27
88         law  9.837741e+26
33           3  9.631431e+26
18        titl  9.280837e+26
39        desc  8.956559e+26
69          ha  8.464311e+26
63        case  8.238007e+26
122      amend  7.718367e+26
424     punish  7.049404e+26
105     insert  6.972722e+26
127     provid  6.744963e+26
156  constitut  6.421402e+26
798      india  6.153688e+26
240      refer  5.934316e+26
85        year  5.852934e+26
420       term  5.755146e+26
22      articl  5.707353e+26
116     public  5.610222e+26
339     offenc  5.527136e+26
56           4  5.512774e+26
421   imprison  5.39

1.3524600368776088e-05
          word   probability
10         ani  2.387457e-07
42         mai  1.031311e-07
72       court  8.282059e-08
41           2  7.494226e-08
51      person  7.459794e-08
174      state  7.056494e-08
46           1  6.749236e-08
37         thi  6.450187e-08
267   sub-sect  6.388758e-08
8        order  5.728864e-08
13      govern  5.682457e-08
88         law  5.369880e-08
80       claus  5.184986e-08
128  substitut  5.069504e-08
33           3  4.780735e-08
18        titl  4.593968e-08
39        desc  4.433452e-08
69          ha  4.190428e-08
63        case  4.114639e-08
122      amend  3.824480e-08
105     insert  3.452305e-08
127     provid  3.396310e-08
424     punish  3.303875e-08
156  constitut  3.179646e-08
798      india  3.158458e-08
85        year  3.102571e-08
240      refer  2.963437e-08
22      articl  2.825084e-08
420       term  2.792962e-08
116     public  2.777170e-08
56           4  2.732679e-08
84     proceed  2.716647e-08
76       befor  2.66

2.1499808014124863e-05
          word   probability
10         ani  3.851155e-07
42         mai  1.616739e-07
72       court  1.519931e-07
41           2  1.183541e-07
51      person  1.171937e-07
46           1  1.066360e-07
8        order  1.054160e-07
37         thi  1.034587e-07
267   sub-sect  1.010492e-07
174      state  9.521364e-08
13      govern  8.988964e-08
80       claus  8.200701e-08
128  substitut  8.019038e-08
88         law  7.626151e-08
33           3  7.544594e-08
18        titl  7.266375e-08
39        desc  7.012483e-08
69          ha  6.649699e-08
63        case  6.449977e-08
122      amend  6.043689e-08
127     provid  5.594179e-08
105     insert  5.460068e-08
798      india  5.150854e-08
156  constitut  5.028258e-08
240      refer  4.646220e-08
424     punish  4.505330e-08
22      articl  4.468494e-08
85        year  4.457324e-08
116     public  4.398757e-08
56           4  4.316194e-08
84     proceed  4.215552e-08
140       word  4.163821e-08
2       applic  4.15

1.4349941421124029e+29
          word   probability
10         ani  2.500359e+27
42         mai  1.114519e+27
174      state  9.043996e+26
72       court  8.732270e+26
41           2  7.990803e+26
51      person  7.444347e+26
46           1  7.184900e+26
37         thi  6.859527e+26
267   sub-sect  6.824703e+26
8        order  6.107332e+26
13      govern  6.070235e+26
88         law  5.986692e+26
80       claus  5.538641e+26
128  substitut  5.401491e+26
33           3  5.092806e+26
18        titl  4.907613e+26
39        desc  4.736138e+26
69          ha  4.475499e+26
63        case  4.355514e+26
122      amend  4.081124e+26
293  establish  3.916723e+26
105     insert  3.686741e+26
127     provid  3.549594e+26
156  constitut  3.395226e+26
420       term  3.324169e+26
798      india  3.225115e+26
240      refer  3.137994e+26
22      articl  3.017959e+26
116     public  2.966529e+26
56           4  2.915074e+26
85        year  2.865933e+26
84     proceed  2.846490e+26
140       word  2.81

7.798879269501245e+78
          word   probability
10         ani  1.403936e+77
42         mai  5.906658e+76
72       court  5.707345e+76
41           2  4.316867e+76
8        order  4.239543e+76
51      person  4.123280e+76
46           1  3.874112e+76
37         thi  3.723880e+76
267   sub-sect  3.673711e+76
174      state  3.504736e+76
13      govern  3.252080e+76
80       claus  2.966612e+76
128  substitut  2.900820e+76
33           3  2.727981e+76
18        titl  2.628610e+76
88         law  2.609205e+76
39        desc  2.536765e+76
69          ha  2.400063e+76
63        case  2.336078e+76
122      amend  2.186814e+76
105     insert  1.994661e+76
127     provid  1.904519e+76
74       power  1.902327e+76
156  constitut  1.821807e+76
798      india  1.752549e+76
2       applic  1.712781e+76
240      refer  1.681872e+76
424     punish  1.674563e+76
85        year  1.637953e+76
22      articl  1.616504e+76
116     public  1.589563e+76
56           4  1.561389e+76
84     proceed  1.529

6.468412694839955e+73
          word   probability
10         ani  1.167375e+72
42         mai  4.880017e+71
72       court  4.609341e+71
41           2  3.564771e+71
51      person  3.513765e+71
8        order  3.387782e+71
46           1  3.228978e+71
37         thi  3.063033e+71
267   sub-sect  3.040291e+71
174      state  2.943760e+71
13      govern  2.704862e+71
80       claus  2.469901e+71
128  substitut  2.427080e+71
88         law  2.377504e+71
33           3  2.268944e+71
18        titl  2.186256e+71
39        desc  2.109867e+71
69          ha  2.013078e+71
63        case  1.943552e+71
122      amend  1.818693e+71
105     insert  1.642999e+71
127     provid  1.627467e+71
798      india  1.540369e+71
156  constitut  1.515671e+71
74       power  1.421056e+71
240      refer  1.398332e+71
424     punish  1.365936e+71
2       applic  1.354083e+71
22      articl  1.345876e+71
85        year  1.336617e+71
116     public  1.322162e+71
76       befor  1.318685e+71
56           4  1.298

In [489]:
out_dict

{'AILA_Q1': [('S173', 59558.525942700224),
  ('S53', 32931.981970312416),
  ('S118', 12601.530395715501),
  ('S122', 141.59821821183738),
  ('S160', -11832.006073538658),
  ('S114', -12663.51402954091),
  ('S110', -13610.25945321664),
  ('S9', -13839.095940469471),
  ('S125', -14222.131037511372),
  ('S3', -14452.116399315883),
  ('S112', -17984.50254678786),
  ('S50', -22820.789912408356),
  ('S79', -23206.819983577338),
  ('S107', -23216.530972470508),
  ('S178', -23889.097957301667),
  ('S155', -24218.26705381588),
  ('S21', -24305.216492013613),
  ('S78', -30878.745798124648),
  ('S131', -36698.55445409634),
  ('S63', -37622.28371341683),
  ('S153', -39290.471844766515),
  ('S68', -42681.23053972883),
  ('S121', -44226.76189131365),
  ('S2', -44385.097508532075),
  ('S23', -49046.70414822587),
  ('S76', -49349.05542711215),
  ('S64', -50025.24774868542),
  ('S103', -50641.169722713414),
  ('S80', -53767.25741326333),
  ('S136', -58718.25425453333),
  ('S4', -64237.29773097816),
  (

In [490]:
dictionary

{'be': 1,
 'action': 1,
 'later': 1,
 'warranto': 1,
 'of': 1,
 'applic': 1,
 'stand': 1,
 'territori': 1,
 'to': 1,
 'or': 1,
 'been': 1,
 'next': 1,
 'in': 1,
 'dispos': 1,
 'confer': 1,
 'petit': 1,
 'order': 1,
 'natur': 1,
 'last': 1,
 'b': 1,
 'that': 1,
 'is': 1,
 'ani': 1,
 'whose': 1,
 'where': 1,
 'issu': 1,
 'those': 1,
 'such': 1,
 'aris': 1,
 'govern': 1,
 'prohibit': 1,
 'expiri': 1,
 'appropri': 1,
 'parti': 1,
 'titl': 1,
 'the': 1,
 'have': 1,
 'so': 1,
 'purpos': 1,
 'on': 1,
 'counsel': 1,
 'week': 1,
 'articl': 1,
 'vacat': 1,
 'exercis': 1,
 'anyth': 1,
 'document': 1,
 'furnish': 1,
 'shall': 1,
 'iii': 1,
 'period': 1,
 'resid': 1,
 'everi': 1,
 'date': 1,
 'for': 1,
 'author': 1,
 '3': 1,
 'not': 1,
 'suprem': 1,
 'throughout': 1,
 'part': 1,
 'seat': 1,
 'give': 1,
 'also': 1,
 'as': 1,
 'whom': 1,
 'stai': 1,
 'make': 1,
 'thi': 1,
 'quo': 1,
 'desc': 1,
 'interim': 1,
 '2': 1,
 'mai': 1,
 'plea': 1,
 'wholli': 1,
 'receiv': 1,
 'which': 1,
 '1': 1,
 'right': 

In [491]:
out_dict

{'AILA_Q1': [('S173', 59558.525942700224),
  ('S53', 32931.981970312416),
  ('S118', 12601.530395715501),
  ('S122', 141.59821821183738),
  ('S160', -11832.006073538658),
  ('S114', -12663.51402954091),
  ('S110', -13610.25945321664),
  ('S9', -13839.095940469471),
  ('S125', -14222.131037511372),
  ('S3', -14452.116399315883),
  ('S112', -17984.50254678786),
  ('S50', -22820.789912408356),
  ('S79', -23206.819983577338),
  ('S107', -23216.530972470508),
  ('S178', -23889.097957301667),
  ('S155', -24218.26705381588),
  ('S21', -24305.216492013613),
  ('S78', -30878.745798124648),
  ('S131', -36698.55445409634),
  ('S63', -37622.28371341683),
  ('S153', -39290.471844766515),
  ('S68', -42681.23053972883),
  ('S121', -44226.76189131365),
  ('S2', -44385.097508532075),
  ('S23', -49046.70414822587),
  ('S76', -49349.05542711215),
  ('S64', -50025.24774868542),
  ('S103', -50641.169722713414),
  ('S80', -53767.25741326333),
  ('S136', -58718.25425453333),
  ('S4', -64237.29773097816),
  (

In [492]:
with open('srm1.pickle', 'wb') as handle:
    pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)